In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

#[example] Load prompt from .json or .yaml files
#prompt = load_prompt("./prompt.json")
#prompt = load_prompt("./prompt.yaml")
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature=0.1)

In [2]:
movie_emoji_examples = [
    {
        "title": "Inside Out 2",
        "answer": """😀😭🏒""",
    },
    {
        "title": "Twisters",
        "answer": """🌎🌪👓""",
    },
    {
        "title": "The Dark Knight",
        "answer": """🦇👮🤡""",
    },
]


In [7]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True,
)

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Please give me only 3 emojis that represent the movie {title}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=movie_emoji_examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you can represent the movie with only three emojis"),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input":question},
        {"output": result.content},
    )
    print(result)

In [8]:
invoke_chain("Forrest Gump")

content='🏃🍫🎸'


In [9]:
invoke_chain("Avengers")

content='🦸\u200d♂️💥🌌'


In [10]:
invoke_chain("What is the title of the movie I asked first?")

content='The title of the movie you asked first is "Forrest Gump."'


### Movie explain chatbot


In [8]:
movie_examples = [
    {
        "title": "Inside Out 2",
        "answer": """
        The Brief information about the movie Inside Out 2 is as below:
        Director: Kelsey Mann
        Main cast: Amy Poehler, Maya Hawke, Kensington Tallman
        Budget: $200,000,000
        Revenue : $1,649,599,772
        Genres: Animation, Family
        Rating: 77%
        Keywords: emotions, joy, sadness, sequel, teenage girl
        """,
    },
    {
        "title": "Twisters",
        "answer": """
        The Brief information about the movie Twisters is as below:
        Director: Lee Isaac Chung
        Main cast: Daisy Edgar-Jones, Glen Powell, Anthony Ramos
        Budget: $155,000,000
        Revenue : $347,273,990
        Genres: Action, Adventure
        Rating: 70%
        Keywords: tornado, twister, climate, weather, natural disaster
        """,
    },
    {
        "title": "The Dark Knight",
        "answer": """
        The Brief information about the movie The Dark Knight is as below:
        Director: Christopher Nolan
        Main cast: Christian Bale, Heath Ledger, Michael Caine
        Budget: $185,000,000
        Revenue : $1,004,558,444
        Genres: Drama, Action
        Rating: 85%
        Keywords: superhero, crime fighter, secret identity, anti hero, joker
        """,
    },
    {
        "title": "Parasite",
        "answer": """
        The Brief information about the movie Parasite is as below:
        Director: Bong Joon-ho
        Main cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong
        Budget: $11,363,000
        Revenue : $257,591,776
        Genres: Comedy, Thriller
        Rating: 85%
        Keywords: class differences, dark comedy, unemployed, family, south korea
        """,
    },
]


In [9]:
chat = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature=0.1, streaming=True)
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Could you give me an information about the movie {title}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=movie_examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, please give me an answer based on the in the same format as the examples."),
        example_prompt,
        ("human", "Could you give me an information about the movie {title}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"title": "Forrest Gump"})


        The Brief information about the movie Forrest Gump is as below:
        Director: Robert Zemeckis
        Main cast: Tom Hanks, Robin Wright, Gary Sinise
        Budget: $55,000,000
        Revenue : $678,226,247
        Genres: Drama, Romance
        Rating: 88%
        Keywords: historical events, slow-witted, love story, vietnam war, feather
        

AIMessageChunk(content='\n        The Brief information about the movie Forrest Gump is as below:\n        Director: Robert Zemeckis\n        Main cast: Tom Hanks, Robin Wright, Gary Sinise\n        Budget: $55,000,000\n        Revenue : $678,226,247\n        Genres: Drama, Romance\n        Rating: 88%\n        Keywords: historical events, slow-witted, love story, vietnam war, feather\n        ')

In [ ]:
# Code sample without chain 
# prompt = template.format_messages(
#     max_item = 10,
#     question = "Whare are the planets?"
# )
# result = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(result.content)

In [6]:
poetry_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative poetry to write about programming languages. Your task is to craft poetic verses that capture the essence, characteristics, and unique qualities of various programming languages."),
    ("human", "Create a poem about {language}"),
])


poetry_chain = poetry_prompt | chat

In [7]:
commentator_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a thoughtful and insightful AI designed to analyze and explain poetry. Your role is to provide clear, detailed explanations of poems, breaking down their structure, themes, literary devices, and the emotions or ideas they convey."),
    ("human", "Read the following poem and provide a detailed explanation. : {poem}"),
])

commentator_chain = commentator_prompt | chat

In [ ]:
final_chain = {"poem": poetry_chain} | commentator_chain

final_chain.invoke({"language": "python"})